In [2]:
import pygame
import sys
import time
from collections import deque
    
def movimiento_en_la_matriz(direccion, eje_x, eje_y, filas, columnas):
    eje_x_actual = eje_x
    eje_y_actual = eje_y
    eje_x += direccion[0]
    eje_y += direccion[1]
    if 0 <= eje_x < filas and 0 <= eje_y < columnas:
        return eje_x, eje_y
    else:
        return eje_x_actual, eje_y_actual

def dibujar_estado(pantalla, matriz, eje_x, eje_y, destino_x, destino_y, filas, columnas, ancho_celda, alto_celda, margen, fuente,camino):
    pantalla.fill((0, 0, 0))
    for i in range(filas):
        for j in range(columnas):
            color = (200, 200, 200)
            if (i,j) in camino:
                color = (0,0, 255)
            if i == eje_x and j == eje_y:
                color = (0, 255, 0)
            elif i == destino_x and j == destino_y:
                color = (255, 0, 0)

            rect = pygame.Rect(j * (ancho_celda + margen), i * (alto_celda + margen), ancho_celda, alto_celda)
            pygame.draw.rect(pantalla, color, rect)
            texto = fuente.render(str(matriz[i][j]), True, (0, 0, 0))
            pantalla.blit(texto, (rect.x + 20, rect.y + 20))
    
    pygame.display.flip()


def Laberinto_Saltarin_Pygame(filas, columnas, fila_i, columna_i, fila_d, columna_d, numeros_matriz):
    pygame.init()
    camino = []

    ancho_celda = 100
    alto_celda = 100
    margen = 5
    ancho = columnas * (ancho_celda + margen)
    alto = filas * (alto_celda + margen)
    pantalla = pygame.display.set_mode((ancho, alto))
    pygame.display.set_caption("Laberinto Saltarín")

    # Matriz
    matriz = [numeros_matriz[i * columnas:(i + 1) * columnas] for i in range(filas)]

    eje_x = fila_i - 1
    eje_y = columna_i - 1
    destino_x = fila_d - 1
    destino_y = columna_d - 1

    pasos = 0
    fuente = pygame.font.SysFont(None, 24)

    reloj = pygame.time.Clock()
    corriendo = True
    juego_ganado = False

    def buscar_camino_valido(x_inicio, y_inicio, x_destino, y_destino):
        nonlocal eje_x, eje_y, pasos

        visitado_local = [[False for _ in range(columnas)] for _ in range(filas)]
        padres = [[None for _ in range(columnas)] for _ in range(filas)]
        cola = deque()
        cola.append((x_inicio, y_inicio))
        visitado_local[x_inicio][y_inicio] = True

        encontrado = False

        while cola:
            x, y = cola.popleft()
            valor = matriz[x][y]
            direcciones = [(-valor, 0), (valor, 0), (0, -valor), (0, valor)]

            for dx, dy in direcciones:
                nx = x + dx
                ny = y + dy
                if 0 <= nx < filas and 0 <= ny < columnas and not visitado_local[nx][ny]:
                    padres[nx][ny] = (x, y)
                    visitado_local[nx][ny] = True
                    cola.append((nx, ny))
                    if nx == x_destino and ny == y_destino:
                        encontrado = True
                        break
            if encontrado:
                break

        if not encontrado:
            return False

        camino_reversa = []
        nodo = (x_destino, y_destino)
        while nodo != (x_inicio, y_inicio):
            camino_reversa.append(nodo)
            nodo = padres[nodo[0]][nodo[1]]
        camino_reversa.reverse()

        for (nx, ny) in camino_reversa:
            eje_x, eje_y = nx, ny
            pasos += 1
            dibujar_estado(pantalla, matriz, eje_x, eje_y, destino_x, destino_y,
                           filas, columnas, ancho_celda, alto_celda, margen, fuente, camino)
            time.sleep(0.5)

        return True
    
    def DFS_agente(x, y, visitado):
        nonlocal eje_x, eje_y, pasos, juego_ganado, camino

        if x < 0 or x >= filas or y < 0 or y >= columnas:
            return False

        if visitado[x][y]:
            return False

        visitado[x][y] = True
        camino.append((x,y))

        eje_x, eje_y = x, y  # Para mostrar visualmente la posición
        dibujar_estado(pantalla, matriz, eje_x, eje_y, destino_x, destino_y, filas, columnas, ancho_celda, alto_celda, margen, fuente,camino)
        time.sleep(2.0)

        if x == destino_x and y == destino_y:
            juego_ganado = True
            return True
        pasos += 1

        valor = matriz[x][y]
        
        direcciones = [
            (-valor, 0),
            (valor, 0),
            (0, -valor),
            (0, valor)
        ]

        for dx, dy in direcciones:
            nuevo_x = x + dx
            nuevo_y = y + dy
            if DFS_agente(nuevo_x, nuevo_y, visitado):
                pasos +=1
                return True
            
        camino.pop()
        visitado[x][y] = False

        if len (camino) > 0:
            destino_retorno = camino[-1]
            retorno_exitoso = buscar_camino_valido(x, y, destino_retorno[0], destino_retorno[1])
            if not retorno_exitoso:
                return False
                
        return False

    
    while corriendo:
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                corriendo = False
            elif evento.type == pygame.KEYDOWN:
                if evento.key == pygame.K_a:
                    visitado = [[False for _ in range(columnas)] for _ in range(filas)]
                    DFS_agente(fila_i - 1, columna_i - 1, visitado)

                if juego_ganado:
                    if evento.key ==pygame.K_r:
                        eje_x = fila_i - 1
                        eje_y = columna_i - 1
                        pasos = 0
                        juego_ganado = False
                else:
                    posicion_actual = matriz[eje_x][eje_y]
                    arriba = (-posicion_actual, 0)
                    abajo = (posicion_actual, 0)
                    izquierda = (0, -posicion_actual)
                    derecha = (0, posicion_actual)

                    movimientos = {
                        pygame.K_UP: arriba,
                        pygame.K_DOWN: abajo,
                        pygame.K_LEFT: izquierda,
                        pygame.K_RIGHT: derecha
                    }

                    if evento.key in movimientos:
                        direccion = movimientos[evento.key]
                        eje_x, eje_y = movimiento_en_la_matriz(direccion, eje_x, eje_y, filas, columnas)
                        pasos += 1
                
        dibujar_estado(pantalla, matriz, eje_x, eje_y, destino_x, destino_y, filas, columnas, ancho_celda, alto_celda, margen, fuente,camino)
        if eje_x == destino_x and eje_y == destino_y:
            juego_ganado = True
            texto_final = fuente.render(f"¡Ganaste en {pasos} pasos!", True, (255, 255, 255))
            pantalla.blit(texto_final, (10, alto - 30))
            reiniciar = fuente.render("Presiona R para intentar nuevamente", True, (255,255,0))
            pantalla.blit(reiniciar,(100, alto - 250))
            pygame.display.flip()
        reloj.tick(30)

    pygame.quit()
    sys.exit()

# --- Entradas de prueba ---
entrada = list(map(int, input("Filas Columnas Fila_i Col_i Fila_d Col_d: ").split()))
filas, columnas, fila_i, columna_i, fila_d, columna_d = entrada
numeros_matriz = list(map(int, input("Numeros de la matriz: ").split()))

Laberinto_Saltarin_Pygame(filas, columnas, fila_i, columna_i, fila_d, columna_d, numeros_matriz)


Filas Columnas Fila_i Col_i Fila_d Col_d:  5 5 1 1 2 4
Numeros de la matriz:  3 4 1 3 1 3 3 3 0 2 3 1 2 2 3 4 2 3 3 3 4 1 4 3 2


SystemExit: 